In [1]:
import numpy as np

import adventofcode

In [2]:
arr = adventofcode.read("../data/day6.txt", to_array=True)

In [3]:
def change_direction(current_direction):
    if (current_direction == np.array([-1, 0])).all():
        return np.array([0, 1])
    elif (current_direction == np.array([0, 1])).all():
        return np.array([1, 0])
    elif (current_direction == np.array([1, 0])).all():
        return np.array([0, -1])
    elif (current_direction == np.array([0, -1])).all():
        return np.array([-1, 0])
    else:
        raise ValueError("Wrong direction")

In [4]:
def is_legal_position(position: tuple, array: np.ndarray) -> bool:
    # In this assignment it is important that outside_bounds is also legal (beacuse it is a goal)
    return adventofcode.is_outside_bounds(position, array.shape) or array[
        *position
    ] in {".", "X"}

In [5]:
def find_new_direction(
    current_position: tuple,
    current_direction: np.ndarray,
    array: np.ndarray,
    seen_directions=0,
):
    if seen_directions == 4:
        raise RuntimeError("Can not find a valid direction")
    direction = current_direction.copy()
    test_position = current_position + direction
    if not is_legal_position(test_position, array):
        direction = change_direction(direction)
        direction = find_new_direction(
            current_position, direction, array, seen_directions=seen_directions + 1
        )

    return direction

In [6]:
def walk_through_array(arr):
    temp_arr = arr.copy()
    current_position = adventofcode.find_unique_value_coords(temp_arr, "^")
    direction = adventofcode.DIRECTIONS["N"]
    while True:
        if adventofcode.is_outside_bounds(current_position, arr.shape):
            break
        temp_arr[*current_position] = "X"

        direction = find_new_direction(current_position, direction, temp_arr)
        current_position += direction

    return temp_arr


walked_array = walk_through_array(arr)

In [7]:
(walked_array == "X").sum()

4374

# Part 2

In [8]:
import tqdm

In [9]:
# The following list-based method is taken from
# It takes only ~40 seconds to run, ~50x faster than my first numpy based method.
# This is surprising, even moreso because the numpy base solution only checks obstacles in the primary path of the guard, and this solution checks every spot, so it is more brute force
# Im figuring out where the time is taken
# (Apparently, running with profiler is much slower (~ 4x). I don't know why)
def main2():
    data = adventofcode.read("../data/day6.txt")
    grid = list(map(list, data.splitlines()))

    rows = len(grid)
    cols = len(grid[0])

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == "^":
                break
        else:
            continue
        break

    def loops(grid, r, c):
        dr = -1
        dc = 0

        seen = set()

        while True:
            seen.add((r, c, dr, dc))
            if r + dr < 0 or r + dr >= rows or c + dc < 0 or c + dc >= cols:
                return False
            if grid[r + dr][c + dc] == "#":
                dc, dr = -dr, dc
            else:
                r += dr
                c += dc
            if (r, c, dr, dc) in seen:
                return True

    count = 0

    for cr in tqdm.tqdm(range(rows)):
        for cc in range(cols):
            if grid[cr][cc] != ".":
                continue
            grid[cr][cc] = "#"
            if loops(grid, r, c):
                count += 1
            grid[cr][cc] = "."
    return count


main2()

100%|██████████| 130/130 [00:35<00:00,  3.68it/s]


1705

In [10]:
def remember_all_states(arr):
    temp_arr = arr.copy()
    current_position = adventofcode.find_unique_value_coords(temp_arr, "^")
    direction = adventofcode.DIRECTIONS["N"]
    states = []
    while True:
        if adventofcode.is_outside_bounds(current_position, arr.shape):
            break
        state = (tuple(current_position), tuple(direction))
        states.append(state)

        direction = find_new_direction(current_position, direction, temp_arr)
        current_position += direction

    return states


def get_option_new_walls(arr) -> set:
    states_original = remember_all_states(arr)
    positions = [state[0] for state in states_original]
    return set(positions)

In [12]:
import tqdm


def main(arr, speedup=1):
    # Speedup was used to test which parts were slow; mostly its the endless out of bounds checking
    options_new_walls = get_option_new_walls(arr)
    starting_position = np.asarray(np.where(arr == "^")).T[0]
    starting_direction = np.array([-1, 0])
    starting_state = (*starting_position, *starting_direction)

    loops = 0
    positions_full_loop = []
    temp_arr = arr.copy()
    for y, x in tqdm.tqdm(list(options_new_walls)[::speedup]):

        if (np.array([y, x]) == starting_position).all():
            continue

        temp_arr[y, x] = "#"
        states = set([starting_state])
        current_position = starting_position.copy()
        direction = starting_direction.copy()

        while True:
            test_position = current_position + direction

            if adventofcode.is_outside_bounds(test_position, arr.shape):
                break

            if temp_arr[*test_position] == "#":
                direction = change_direction(direction)

                test_position = current_position + direction
                if adventofcode.is_outside_bounds(test_position, arr.shape):
                    break
                if temp_arr[*test_position] == "#":
                    direction = change_direction(direction)

                # This can only happen twice - there is no way to run into a square that has walls on 4 sides
                # or on 3 sides from your direction (I suppose technically if it is on the bottom of the starting position)

            current_position += direction
            state = (*current_position, *direction)
            if state in states:
                loops += 1
                positions_full_loop.append((x, y))

                break
            states.add(state)
        temp_arr[y, x] = "."

    return positions_full_loop


poss = main(arr)
print(len(set(poss)))

100%|██████████| 4374/4374 [02:16<00:00, 32.08it/s]

1705


In [ ]:
%load_ext line_profiler
%lprun -f main main(arr, speedup=10)